In [1]:
import os 
os.chdir("../")

In [2]:
%pwd

'd:\\Project\\Kidney_Disease_Classification\\End_to_end_Kidney_Disease_Classification'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [10]:
os.environ["MLFLOW_TRACKING_PASSWORD"]="99aa7f2e054ba604a83227723b517dab3b6131ba"
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Prasanth-Gururaj/End_to_end_Kidney_Disease_Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Prasanth-Gururaj"

In [11]:
import dagshub
dagshub.init(repo_owner='Prasanth-Gururaj', repo_name='End_to_end_Kidney_Disease_Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2025-04-01 14:22:26,377: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 401 Unauthorized"]


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Prasanth\anaconda3\envs\kidney\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1347a46b-655c-4daf-99ca-574efccb966f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a43b87cfe89e57af4c11ed03209eefe708ce7f6bd2c8d82f38b3d8384342cda6


[2025-04-01 14:22:35,695: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-04-01 14:22:36,071: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-04-01 14:22:36,382: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


...

[2025-04-01 14:22:36,387: INFO: helpers: Accessing as Prasanth-Gururaj]
[2025-04-01 14:22:36,748: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Prasanth-Gururaj/End_to_end_Kidney_Disease_Classification "HTTP/1.1 200 OK"]
[2025-04-01 14:22:37,063: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Prasanth-Gururaj/End_to_end_Kidney_Disease_Classification"

[2025-04-01 14:22:37,071: INFO: helpers: Initialized MLflow to track repo "Prasanth-Gururaj/End_to_end_Kidney_Disease_Classification"]


Repository Prasanth-Gururaj/End_to_end_Kidney_Disease_Classification initialized!

[2025-04-01 14:22:37,073: INFO: helpers: Repository Prasanth-Gururaj/End_to_end_Kidney_Disease_Classification initialized!]


In [13]:
os.environ["MLFLOW_TRACKING_PASSWORD"]="99aa7f2e054ba604a83227723b517dab3b6131ba"

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        # Get the MLflow tracking URI
        tracking_uri = mlflow.get_tracking_uri()
        tracking_url_type_store = tracking_uri.split(":")[0]
        
        # Log model - always do this regardless of tracking store type
        try:
            # For remote tracking servers (not file-based)
            if tracking_url_type_store != "file" and not tracking_uri.startswith("file:"):
                try:
                    # Try to register the model, but catch permission errors
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                except mlflow.exceptions.MlflowException as registry_error:
                    # If we get a permission error, just log without registering
                    print(f"Warning: Could not register model due to: {registry_error}")
                    print("Logging model without registration instead.")
                    mlflow.keras.log_model(self.model, "model")
            else:
                # For file-based tracking, don't try to register
                print("Using file-based tracking. Model registry not available - logging without registration.")
                mlflow.keras.log_model(self.model, "model")
                
            # Log other metrics and parameters as needed
            # mlflow.log_metric("accuracy", self.metrics["accuracy"])
            # etc.
            
        except Exception as e:
            print(f"Error in logging to MLflow: {e}")
            raise

In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-04-01 14:26:57,974: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-01 14:26:57,976: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-01 14:26:57,977: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 6s 707ms/step - loss: 23.0048 - accuracy: 0.4820
[2025-04-01 14:27:04,727: INFO: common: json file saved at: scores.json]


2025/04/01 14:27:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-04-01 14:27:05,568: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Prasanth\AppData\Local\Temp\tmpn9z77j9b\model\data\model\assets
[2025-04-01 14:27:05,886: INFO: builder_impl: Assets written to: C:\Users\Prasanth\AppData\Local\Temp\tmpn9z77j9b\model\data\model\assets]


2025/04/01 14:27:24 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Logging model without registration instead.
[2025-04-01 14:27:25,278: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Prasanth\AppData\Local\Temp\tmpftpg9xhr\model\data\model\assets
[2025-04-01 14:27:25,911: INFO: builder_impl: Assets written to: C:\Users\Prasanth\AppData\Local\Temp\tmpftpg9xhr\model\data\model\assets]


In [33]:
import os

os.environ["MLFLOW_TRACKING_USERNAME"] = "Prasanth-Gururaj"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f4bafe26a567e79259be7b0b621c02b4e2e82245"
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Prasanth-Gururaj/End_to_end_Kidney_Disease_Tracking"

print("MLflow Tracking URI:", os.environ.get("MLFLOW_TRACKING_URI"))
print("MLflow Username:", os.environ.get("MLFLOW_TRACKING_USERNAME"))
print("MLflow Password:", os.environ.get("MLFLOW_TRACKING_PASSWORD"))  # Just to verify, remove after testing



MLflow Tracking URI: https://dagshub.com/Prasanth-Gururaj/End_to_end_Kidney_Disease_Tracking
MLflow Username: Prasanth-Gururaj
MLflow Password: f4bafe26a567e79259be7b0b621c02b4e2e82245
